In [ ]:
import numpy as np
import pandas as pd
import nd2reader
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews.operation.datashader import regrid
import skimage.filters
import skimage.feature
import scipy.ndimage
import peakutils
from tqdm import tnrange, tqdm_notebook
import dask
import dask.array as da
import distributed
from distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
from cytoolz import partial, compose
from itertools import repeat
from glob import glob
import cachetools
import numpy_indexed
import pickle
import pyarrow as pa
import warnings
import os
from numbers import Integral
from dask.delayed import Delayed

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from segmentation import *
# from util import *
# from matriarch_stub import *
import segmentation
import matriarch_stub

In [ ]:
plt.rcParams["figure.figsize"] = (8, 8)
hv.extension("bokeh")

# Load data

In [ ]:
# filename = '/n/groups/paulsson/jqs1/molecule-counting/190522photobleaching_noflatcorr_test1.pickle'
filename = (
    "/n/groups/paulsson/jqs1/molecule-counting/190521photobleaching_noflatcorr.pickle"
)

In [ ]:
data = pickle.load(open(filename, "rb"))

# Debug

In [ ]:
nd2_filename = "/n/scratch2/jqs1/190508/GFP_100ms_20pct.nd2"
channel = "FITC"

In [ ]:
idx = 920
pos = 0
regionprops = data[nd2_filename][pos]["regionprops"]
seg_frame = np.asarray(data[nd2_filename][pos]["segmentation_frame"])
label_frame = np.asarray(data[nd2_filename][pos]["labels"])
y, x = regionprops.iloc[idx, :][["centroid_x", "centroid_y"]]
s = 10
x_lim = y_lim = 2048
x, y = int(x), int(y)
x_min = max(x - s, 0)
x_max = min(x + s, x_lim)
y_min = max(y - s, 0)
y_max = min(y + s, y_lim)
t_max = data[nd2_filename][pos]["traces"]["mean"].shape[1]
seg = seg_frame[y_min:y_max, x_min:x_max]
labels = label_frame[y_min:y_max, x_min:x_max]

In [ ]:
%%output max_frames=2000
hv.HoloMap(
    {
        t: hv.Image(
            segmentation.get_nd2_frame(nd2_filename, 0, channel, t)[
                y_min:y_max, x_min:x_max
            ]
        )
        for t in range(0, t_max // 10, 1)
    }
)

In [ ]:
plt.plot(np.asarray(data[nd2_filename][0]["traces"]["mean"]).T[:, 920])

In [ ]:
centroid_trace = np.array(
    [
        segmentation.get_nd2_frame(nd2_filename, 0, channel, t)[y, x]
        for t in range(0, t_max, 1)
    ]
)
centroid_trace2 = np.array(
    [
        segmentation.get_nd2_frame(nd2_filename, 0, channel, t)[y + 1, x + 1]
        for t in range(0, t_max, 1)
    ]
)
bg_trace = np.array(
    [
        segmentation.get_nd2_frame(nd2_filename, 0, channel, t)[y + 10, x + 10]
        for t in range(0, t_max, 1)
    ]
)

In [ ]:
plt.plot(centroid_trace)
plt.plot(centroid_trace2)
plt.plot(bg_trace)
plt.plot(np.asarray(data[nd2_filename][0]["traces"]["mean"]).T[:, 1])
plt.plot(np.asarray(data[nd2_filename][0]["traces"]["mean"]).T[:, 500])

In [ ]:
plt.plot(
    np.asarray(
        data["/n/scratch2/jqs1/190507/mkate_100ms_50pct.nd2"][0]["traces"]["mean"]
    ).T[:, :5]
)

In [ ]:
plt.plot(
    np.asarray(
        data["/n/scratch2/jqs1/190507/mkate_100ms_50pct.nd2"][0]["traces"]["median"]
    ).T[:, :10]
)

In [ ]:
%output size=200
hv.Image(
    np.asarray(
        data["/n/scratch2/jqs1/190411/Noah_Runs/GFP_100ms_100pct_0001.nd2"][0][
            "segmentation_frame"
        ]
    )
)

In [ ]:
nd2 = segmentation.get_nd2_reader(
    "/n/scratch2/jqs1/190411/Noah_Runs/GFP_100ms_100pct_0001.nd2"
)

In [ ]:
%output size=200
hv.HoloMap({t: hv.Image(nd2.get_frame_2D(v=0, t=t)) for t in range(5)})

In [ ]:
data_graph["/n/scratch2/jqs1/190411/Noah_Runs/GFP_100ms_100pct_0001.nd2"][0].keys()